## Import packages

In [1]:
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from skimage.io import imread

In [2]:
print(f'torch=={torch.__version__}')
import skimage
print(f'skimage=={skimage.__version__}')
print(f'pandas=={pd.__version__}')

torch==1.12.1+cu102
skimage==0.19.3
pandas==1.4.4


## Necessary paths

In [3]:
project_path = Path.cwd().parent
print(f'project_path = {project_path}')
mri_png_dir = project_path/'data/processed/mri'
print(f'train, val and test MRI images are in {mri_png_dir}')
csv_dir = project_path/'data/processed/csv'
print(f'needed csv files are in {csv_dir}')

project_path = /home/dk/Desktop/projects/multimodal-healthcare/mri
train, val and test MRI images are in /home/dk/Desktop/projects/multimodal-healthcare/mri/data/processed/mri
needed csv files are in /home/dk/Desktop/projects/multimodal-healthcare/mri/data/processed/csv


## Process output conditions

In [4]:
train_df = pd.read_csv(csv_dir/'train_patients.csv')
# only keep necessary columns
cols = ['patient', 'heart_failure', 'coronary_heart', 'myocardial_infarction', 'stroke', 'cardiac_arrest']
train_df = train_df[cols]
train_df

,patient,heart_failure,coronary_heart,myocardial_infarction,stroke,cardiac_arrest
0,c2a58661-9352-1929-fc29-c51fad4c6950,True,False,False,False,False
1,adf80417-f70b-2fe8-a2da-0fdcf70b2509,False,True,False,True,False
2,6cb86432-5f9e-6f74-2289-a8295e84c64e,False,True,False,True,False
3,b8e7ebe2-ef2c-5562-7390-e216389cdba8,False,True,False,True,False
4,fbb75ebb-8b10-2a28-5634-3b8f4da7442b,True,True,False,False,False
...,...,...,...,...,...,...
761,9996a530-4e4a-3e49-ff16-d52837fd3b49,True,False,False,True,False
762,e7a9a523-6df7-6fcb-9496-4380f13c9b71,False,False,False,False,True
763,79ff8a86-85fb-6ff3-1c2e-6b7a2d04d240,False,False,False,False,True
764,13ff227e-4038-4634-2eb2-e64dc710c9b3,False,False,False,True,False


Embed `False` and `True` into `0` and `1`

In [5]:
labels = ['heart_failure', 'coronary_heart', 'myocardial_infarction', 'stroke', 'cardiac_arrest']
train_df[labels] = train_df[labels].astype(int)
train_df

,patient,heart_failure,coronary_heart,myocardial_infarction,stroke,cardiac_arrest
0,c2a58661-9352-1929-fc29-c51fad4c6950,1,0,0,0,0
1,adf80417-f70b-2fe8-a2da-0fdcf70b2509,0,1,0,1,0
2,6cb86432-5f9e-6f74-2289-a8295e84c64e,0,1,0,1,0
3,b8e7ebe2-ef2c-5562-7390-e216389cdba8,0,1,0,1,0
4,fbb75ebb-8b10-2a28-5634-3b8f4da7442b,1,1,0,0,0
...,...,...,...,...,...,...
761,9996a530-4e4a-3e49-ff16-d52837fd3b49,1,0,0,1,0
762,e7a9a523-6df7-6fcb-9496-4380f13c9b71,0,0,0,0,1
763,79ff8a86-85fb-6ff3-1c2e-6b7a2d04d240,0,0,0,0,1
764,13ff227e-4038-4634-2eb2-e64dc710c9b3,0,0,0,1,0


Check if any values are `NaN`

In [6]:
check_nan = train_df.isnull().values.any()
 # printing the result
print(check_nan)

False


In [7]:
train_df['label']= train_df[labels].values.tolist()
train_df

,patient,heart_failure,coronary_heart,myocardial_infarction,stroke,cardiac_arrest,label
0,c2a58661-9352-1929-fc29-c51fad4c6950,1,0,0,0,0,"[1, 0, 0, 0, 0]"
1,adf80417-f70b-2fe8-a2da-0fdcf70b2509,0,1,0,1,0,"[0, 1, 0, 1, 0]"
2,6cb86432-5f9e-6f74-2289-a8295e84c64e,0,1,0,1,0,"[0, 1, 0, 1, 0]"
3,b8e7ebe2-ef2c-5562-7390-e216389cdba8,0,1,0,1,0,"[0, 1, 0, 1, 0]"
4,fbb75ebb-8b10-2a28-5634-3b8f4da7442b,1,1,0,0,0,"[1, 1, 0, 0, 0]"
...,...,...,...,...,...,...,...
761,9996a530-4e4a-3e49-ff16-d52837fd3b49,1,0,0,1,0,"[1, 0, 0, 1, 0]"
762,e7a9a523-6df7-6fcb-9496-4380f13c9b71,0,0,0,0,1,"[0, 0, 0, 0, 1]"
763,79ff8a86-85fb-6ff3-1c2e-6b7a2d04d240,0,0,0,0,1,"[0, 0, 0, 0, 1]"
764,13ff227e-4038-4634-2eb2-e64dc710c9b3,0,0,0,1,0,"[0, 0, 0, 1, 0]"


Only keep the `patient` and the `label` columns

In [8]:
train_patient_label_df = train_df[['patient', 'label']]
train_patient_label_df

,patient,label
0,c2a58661-9352-1929-fc29-c51fad4c6950,"[1, 0, 0, 0, 0]"
1,adf80417-f70b-2fe8-a2da-0fdcf70b2509,"[0, 1, 0, 1, 0]"
2,6cb86432-5f9e-6f74-2289-a8295e84c64e,"[0, 1, 0, 1, 0]"
3,b8e7ebe2-ef2c-5562-7390-e216389cdba8,"[0, 1, 0, 1, 0]"
4,fbb75ebb-8b10-2a28-5634-3b8f4da7442b,"[1, 1, 0, 0, 0]"
...,...,...
761,9996a530-4e4a-3e49-ff16-d52837fd3b49,"[1, 0, 0, 1, 0]"
762,e7a9a523-6df7-6fcb-9496-4380f13c9b71,"[0, 0, 0, 0, 1]"
763,79ff8a86-85fb-6ff3-1c2e-6b7a2d04d240,"[0, 0, 0, 0, 1]"
764,13ff227e-4038-4634-2eb2-e64dc710c9b3,"[0, 0, 0, 1, 0]"


## Get the paths of the MRI `.png` files and give them the right output 

Loop through `train_patient_label_df` dataframe. For each patient, find the paths to the MRI `.png` images by his id

In [9]:
train_mri_dir = mri_png_dir/'train'
train_png_paths = train_mri_dir.glob('*')

In [10]:
train_png_paths = list(train_png_paths)
len(train_png_paths)

20470

Reminder from `data_process.ipynb`: There are only 178 patients in the train split, 62 patients in the val split and 57 patients in the test split that have MRI brain scans. For a patient with a MRI scan, we chose 115 MRI slices => each patient has 115 MRI png images, e.g. 20470 / 178 = 115

In [11]:
def find_mri_png_paths(patient_id: str, mri_png_paths: list) -> list[Path]:
    """
    For each patient, find the corresponding paths to the MRI .png images
    and return them in a list
    """
    res = []
    for path in mri_png_paths:
        if patient_id in path.name:
            res.append(path)
    return res

In [21]:
train_png_mri_label = []
for i, row in train_patient_label_df.iterrows():
    patient_id = row['patient']
    label = row['label']
    png_paths = find_mri_png_paths(patient_id, train_png_paths)
    for png_path in png_paths:
        train_png_mri_label.append((png_path, label))

In [28]:
train_mri_df = pd.DataFrame(train_png_mri_label, columns=['mri_png_path', 'label'])

In [29]:
train_mri_df.to_csv(csv_dir/'train_mri.csv')

## Do the same process above for the val and test split

In [ ]:
val_df = pd.read_csv(csv_dir/'val_patients.csv')
# only keep necessary columns
cols = ['patient', 'heart_failure', 'coronary_heart', 'myocardial_infarction', 'stroke', 'cardiac_arrest']
val_df = val_df[cols]
val_df